<a href="https://colab.research.google.com/github/mikiya2014/My-projects/blob/main/ABtesting_Udacity_HomePage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [40]:
df=pd.read_csv('/content/drive/MyDrive/Projects - Yu (Miki) Wang/Data/homepage_actions.csv')

In [3]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
len(df)

8188

In [41]:
df=df.sort_values('id')

In [42]:
df.reset_index(inplace=True)

In [43]:
df

,index,timestamp,id,group,action
0,4390,2016-11-26 14:38:47.682931,182988,experiment,view
1,5639,2016-12-14 13:19:00.940677,182994,control,view
2,5640,2016-12-14 13:20:29.564080,182994,control,click
3,376,2016-09-29 23:04:42.722325,183089,control,view
4,6568,2016-12-27 10:13:58.808014,183136,experiment,view
...,...,...,...,...,...
8183,7389,2017-01-07 11:50:17.313666,937108,control,view
8184,5254,2016-12-09 07:05:04.965876,937139,experiment,view
8185,5255,2016-12-09 07:06:05.219267,937139,experiment,click
8186,3494,2016-11-14 08:01:25.774204,937217,control,click


In [44]:
df1=df[['id','group']].drop_duplicates()

In [46]:
counts=df1['id'].value_counts()
dup=counts[counts>1].index
dup
# No duplicate id in df1

Int64Index([], dtype='int64')

In [52]:
convert=[]
for i in df1['id']:
  if 'click' in df[df['id']==i]['action'].values:
    convert.append(1)
  else:
    convert.append(0)
df1['convert']=convert

In [53]:
df1

,id,group,convert
0,182988,experiment,0
1,182994,control,1
3,183089,control,0
4,183136,experiment,0
5,183141,experiment,1
...,...,...,...
8181,937003,control,0
8182,937073,control,0
8183,937108,control,0
8184,937139,experiment,1


In [54]:
control=df1[df1['group']=='control']
experiment=df1[df1['group']=='experiment']

In [56]:
current_rate=len(control[control['convert']==1])/(len(control))
current_rate

0.2797118847539016

Hypothesis

H0: experiment group does not have a different performance with control group.

H1: experiment group has a different performance with control group.

Critical value: 0.05.

Performance improvement goal: 5%

Conversion: from view to click=1, only view=0

In [57]:
# Find out the effective sample size that required by the business goal.
effect=sms.proportion_effectsize(0.28,0.33)
required_size=sms.NormalIndPower().solve_power(effect,power=0.8,alpha=0.5,ratio=1)
print(required_size)
# 361 records needed.But we have 6328 records.

360.2943587643993


In [58]:
n_control=len(control)
n_experiment=len(experiment)
converted_control=control['convert'].sum()
converted_experiment=experiment['convert'].sum()

In [60]:
z_stat,pval=proportions_ztest([converted_control,converted_experiment],[n_control,n_experiment])
(lower_con,lower_ex),(upper_con,upper_ex)=proportion_confint([converted_control,converted_experiment],[n_control,n_experiment],alpha=0.05)
print('z-stat:', z_stat)
print('p-value:', pval)
print(f'confidence interval for control: [{lower_con},{upper_con}]')
print(f'confidence interval for experiment: [{lower_ex},{upper_ex}]')

z-stat: -2.618563885349469
p-value: 0.008830075576595804
confidence interval for control: [0.26447121056492595,0.2949525589428772]
confidence interval for experiment: [0.2931892146752537,0.3263034422005807]


P-value is smaller than 0.05 which indicates the null hypothesis can be reject. The confidence interval proved experiment group perfroms better than control group and very close to the aimed convertion rate at 33%. Thus we can conclude the experiment group uplifted the control group by almost 5%.